In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
! pip install wordcloud
from wordcloud import WordCloud, STOPWORDS
from PIL import Image

%matplotlib inline

In [3]:
df=pd.read_csv('file.csv')
df

,Unnamed: 0,tweets,labels
0,0,ChatGPT: Optimizing Language Models for Dialog...,neutral
1,1,"Try talking with ChatGPT, our new AI system wh...",good
2,2,ChatGPT: Optimizing Language Models for Dialog...,neutral
3,3,"THRILLED to share that ChatGPT, our new model ...",good
4,4,"As of 2 minutes ago, @OpenAI released their ne...",bad
...,...,...,...
219289,219289,Other Software Projects Are Now Trying to Repl...,bad
219290,219290,I asked #ChatGPT to write a #NYE Joke for SEOs...,good
219291,219291,chatgpt is being disassembled until it can onl...,bad
219292,219292,2023 predictions by #chatGPT. Nothing really s...,bad


In [3]:
df.drop(labels='Unnamed: 0',axis=1,inplace=True)

In [4]:
# Remove all the tweet links since they all begin with https:
df['tweet_list'] = df['tweets'].str.split('https:')

In [5]:
# Select the text part of the list

text = [i[0] for i in df.tweet_list]
df['text'] = text
df = df[['text', 'labels']]

In [6]:
# Import re for string processing
import re

In [7]:
# Remove all non-alphanumeric characters from the text list

string = r'[A-Za-z0-9 ]'

trim_list=[]

for row in text:
    s=''
    for letter in row:
        if bool(re.match(string, letter)):
            s+=letter
    trim_list.append(s)

In [8]:
# Remove the non-printing characters from text

rep_list = ['\U0001fae1', '\\n', '@', '#', '\xa0', '***', 'https', 'nhttps']

for i in trim_list:
    for j in rep_list:
        if j in i:
            i.replace(j,'')

In [9]:
df['text'] = trim_list

/var/folders/hf/6zg0sbl14sschp84hqddb7k00000gn/T/ipykernel_25248/979011282.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['text'] = trim_list


In [10]:
df.head()

,text,labels
0,ChatGPT Optimizing Language Models for Dialogue,neutral
1,Try talking with ChatGPT our new AI system whi...,good
2,ChatGPT Optimizing Language Models for Dialogue,neutral
3,THRILLED to share that ChatGPT our new model o...,good
4,As of 2 minutes ago OpenAI released their new ...,bad


In [11]:
df.columns

Index(['text', 'labels'], dtype='object')

describe and visualize data

In [12]:
df.describe(include='object')

,text,labels
count,219294,219294
unique,194512,3
top,,bad
freq,872,107796


In [13]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Load dataset into a pandas dataframe
df = pd.read_csv('file.csv')
# Check for missing values
print('Missing values:\n', df.isnull().sum())
# Print summary statistics
print('Summary statistics:\n', df.describe())

Missing values:
 Unnamed: 0    0
tweets        0
labels        0
dtype: int64
Summary statistics:
           Unnamed: 0
count  219294.000000
mean   109646.500000
std     63304.869303
min         0.000000
25%     54823.250000
50%    109646.500000
75%    164469.750000
max    219293.000000


In [14]:
df.columns

Index(['Unnamed: 0', 'tweets', 'labels'], dtype='object')

Models selection

#1 Text Pre-Processing

In [4]:
import nltk
nltk.download('punkt') # downloads you a model
nltk.download('stopwords') 
from nltk.corpus import stopwords
stop = set(stopwords.words('english'))
# print(stop)
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.stem import PorterStemmer 
ps = PorterStemmer() 

# return a list of tokens
def pre_processing_by_nltk(doc, stemming = True, need_sent = False):
    # step 1: get sentences
    sentences = sent_tokenize(doc)
    # step 2: get tokens
    tokens = []
    for sent in sentences:
        words = word_tokenize(sent)
        # step 3 (optional): stemming
        if stemming:
            words = [ps.stem(word) for word in words]
        if need_sent:
            tokens.append(words)
        else:
            tokens += words
    return [w.lower() for w in tokens if w.lower() not in stop]

[nltk_data] Downloading package punkt to /Users/jennyli/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/jennyli/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
df.tweets.apply(pre_processing_by_nltk)

0         [chatgpt, :, optim, languag, model, dialogu, h...
1         [tri, talk, chatgpt, ,, new, ai, system, optim...
2         [chatgpt, :, optim, languag, model, dialogu, h...
3         [thrill, share, chatgpt, ,, new, model, optim,...
4         [2, minut, ago, ,, @, openai, releas, new, cha...
                                ...                        
219289    [softwar, project, tri, replic, chatgpt, http,...
219290    [ask, #, chatgpt, write, #, nye, joke, seo, de...
219291                [chatgpt, disassembl, onli, dissembl]
219292    [2023, predict, #, chatgpt, ., noth, realli, s...
219293    [chatgpt, ,, neat, stuff, http, :, //t.co/qjju...
Name: tweets, Length: 219294, dtype: object

Testing Zip'f Law

#2 Bag Of Words

In [6]:
from tqdm import tqdm
from collections import defaultdict
freq = defaultdict(int)
DF = defaultdict(float)
for doc in tqdm(df.tweets):
    tokens = pre_processing_by_nltk(doc)
    for token in set(tokens):
        DF[token] += 1

100%|█████████████████████████████████| 219294/219294 [01:49<00:00, 1997.35it/s]


In [7]:
from math import log
IDF, vocab = dict(), dict()
for token in DF:
    if DF[token] < 50:
        # this becomes an unk
        pass
    else:
        vocab[token] = len(vocab)
        IDF[token] = log(1 + len(df.tweets) / DF[token])
print(len(DF), len(vocab))

314835 4157


In [8]:
IDF['<UNK>'] = 1
vocab['<UNK>'] = len(vocab)
print(vocab)

{'http': 0, 'chatgpt': 1, 'dialogu': 2, 'optim': 3, '@': 4, ':': 5, 'languag': 6, 'openai': 7, 'model': 8, 'talk': 9, 'ai': 10, 'improv': 11, ',': 12, '.': 13, 'help': 14, 'us': 15, 'new': 16, 'system': 17, 'tri': 18, 'feedback': 19, 'machinelearn': 20, 'datasci': 21, 'amp': 22, '#': 23, 'artificialintelligence\\n\\ntrend': 24, 'via': 25, 'granola': 26, '&': 27, 'identifi': 28, 'articl': 29, 'elbash': 30, ';': 31, 'machine-driven': 32, 'bot': 33, 'rss': 34, 'ramsey': 35, 'digest': 36, 'ai/ml': 37, 'thrill': 38, 'public': 39, 'everyon': 40, 'free': 41, 'dialog': 42, 'share': 43, 'access': 44, '👇': 45, 'ago': 46, '2': 47, 'use': 48, 'releas': 49, 'right': 50, '\\n\\nand': 51, 'minut': 52, 'launch': 53, '\\n': 54, '\\nhttp': 55, 'refus': 56, 'someon': 57, 'come': 58, 'strong': 59, 'provid': 60, 'agre': 61, 'gpt': 62, 'last': 63, 'excit': 64, 'base': 65, 'realli': 66, 'thi': 67, 'coupl': 68, '3': 69, "'ve": 70, "'m": 71, "'s": 72, 'build': 73, 'deploy': 74, 'thing': 75, 'chatbot': 76, 'mon

In [9]:
def tfidf_feature_extractor(doc, vocab, IDF):
    tokens = pre_processing_by_nltk(doc)
    for i, token in enumerate(tokens):
        if token not in vocab:
            tokens[i] = '<UNK>'
    TF = defaultdict(int)
    for token in tokens:
        TF[token] += 1
    x = [0] * len(vocab)
    for token in set(tokens):
        tfidf = log(TF[token] + 1) * IDF[token]
        token_id = vocab[token]
#         print(token, TF[token], IDF[token])
        x[token_id] = tfidf 
    return x

x = tfidf_feature_extractor('tweets', vocab, IDF)
print(x)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [10]:
X = []
for doc in tqdm(df.tweets):
    X.append(tfidf_feature_extractor(doc, vocab, IDF))
y = list(df.labels)

100%|█████████████████████████████████| 219294/219294 [02:35<00:00, 1407.05it/s]


binary-valued vector 

In [11]:
def if_exist_vocab(doc, vocab):
    tokens = pre_processing_by_nltk(doc)  
    for i, token in enumerate(tokens): 
         if token not in vocab:
            tokens[i] = '<UNK>'
    x = [0] * len(vocab)    
    for token in set(tokens):  
        token_id = vocab[token]
        x[token_id] = 1
    return x
X = []
for doc in tqdm(df.tweets):
    X.append(if_exist_vocab(doc, vocab))

100%|█████████████████████████████████| 219294/219294 [02:24<00:00, 1519.59it/s]


In [12]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.2, shuffle=True)

In [14]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA

In [15]:
# create LDA model
model = LDA()

# fit the model to the training data
model.fit(X_train, y_train)


LinearDiscriminantAnalysis()

In [24]:
# make predictions on the test data
y_pred = model.predict(X_test)

# evaluate the performance of the model
from sklearn.metrics import accuracy_score, f1_score, recall_score

accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average='macro')
recall = recall_score(y_test, y_pred, average='macro')

print('Accuracy:', accuracy)
print('F1 Score:', f1)
print('Recall:', recall)


Accuracy: 0.8051255158576347
F1 Score: 0.7761965098471034
Recall: 0.7649138735639793


In [22]:
from sklearn.metrics import f1_score, recall_score

f1 = f1_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')


In [25]:
def tf(doc, vocab):
    tokens = pre_processing_by_nltk(doc)
    for i, token in enumerate(tokens):
        if token not in vocab:
            tokens[i] = '<UNK>'
    x = [0] * len(vocab)
    for token in tokens:
        token_id = vocab[token]
        x[token_id] += 1   
    return x
X = []
for doc in tqdm(df.tweets):
    X.append(tf(doc, vocab))

100%|█████████████████████████████████| 219294/219294 [02:49<00:00, 1290.44it/s]


In [26]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.2, shuffle=True)

In [31]:
# create LDA model
model = LDA()

# fit the model to the training data
model.fit(X_train, y_train)

# make predictions on the test data
y_pred = model.predict(X_test)

# evaluate the performance of the model
from sklearn.metrics import accuracy_score, f1_score, recall_score

accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average='macro')
recall = recall_score(y_test, y_pred, average='macro')

print('Accuracy:', accuracy)
print('F1 Score:', f1)
print('Recall:', recall)

TypeError: A sparse matrix was passed, but dense data is required. Use X.toarray() to convert to a dense numpy array.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(strip_accents=None,
                        lowercase=True,preprocessor=None,
                        tokenizer=pre_processing_by_nltk,
                        use_idf=True,
                        norm='l2',
                        smooth_idf=True)

In [ ]:
X = tfidf.fit_transform(df.tweets)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.2, shuffle=True)

In [ ]:
def tf(doc, vocab):
    tokens = pre_processing_by_nltk(doc)
    for i, token in enumerate(tokens):
        if token not in vocab:
            tokens[i] = '<UNK>'
    x = [0] * len(vocab)
    for token in tokens:
        token_id = vocab[token]
        x[token_id] += 1   
    return x
X = []
for doc in tqdm(df.tweets):
    X.append(tf(doc, vocab))

In [ ]:
# create LDA model
model = LDA()

# fit the model to the training data
model.fit(X_train, y_train)

# make predictions on the test data
y_pred = model.predict(X_test)

# evaluate the performance of the model
from sklearn.metrics import accuracy_score, f1_score, recall_score

accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average='macro')
recall = recall_score(y_test, y_pred, average='macro')

print('Accuracy:', accuracy)
print('F1 Score:', f1)
print('Recall:', recall)